In [4]:
df_QUERY_notasExport = pd.read_excel(
    'uploads//nota_export.xlsx')

In [5]:
from flask import Flask, request, render_template
from flask import Flask, render_template, send_from_directory, request, redirect, url_for, Response, jsonify
import os
from zipfile import ZipFile
import shutil
import subprocess
from werkzeug.utils import secure_filename
import pandas as pd
import numpy as np
import time
from datetime import datetime
import traceback

In [6]:
try:
    print("leitura df_QUERY_z369")
    df_QUERY_z369 = pd.read_json(r'files\df_stage_Nota.json', lines=True)
    df_QUERY_z369['Aux_Status'] = df_QUERY_z369['Aux_Status'].replace('Pendente', 'MSPN')
    df_QUERY_z369['Aux_Status'] = df_QUERY_z369['Aux_Status'].replace('Em processamento', 'MSPR')

# --------------------------------
    try:
        # Lista de novos cabeçalhos
        colunas = [
            "Data da nota", "Hora da nota", "Equipam.", "Parada", "Notificador", "TpNota",
            "CódFalha", "Descrição", "Texto do item", "Texto da causa", "Status do sistema",
            "Nota", "Centro trab.respons.", "Ordem"
        ]

        # Carregar o arquivo
        arquivo = r'uploads//nota_export.xlsx'
        df_QUERY_notasExport = pd.read_excel(arquivo, header=0)

        # Verifique se alguma coluna contém "Unnamed"
        if any("Unnamed" in col for col in df_QUERY_notasExport.columns):
            # Substitua pelos novos cabeçalhos
            df_QUERY_notasExport.columns = colunas
            print(
                "Alguma coluna continha 'Unnamed'. Cabeçalhos substituídos com sucesso!")

        else:
            print("Nenhuma coluna contém 'Unnamed'. Mantendo o DataFrame original.")

    except Exception as e:
        # Captura qualquer erro e retorna uma mensagem personalizada
        error_message = f" PARA CORRIGIR SALVE O ARQUIVO ANEXADO NOVAMENTE, COM 'SALVAR COMO' NO SEU COMPUTADOR e ANEXE NOVAMENTE.Ocorreu um erro ao processar o arquivo: {str(e)}."
        print(error_message)
        
# --------------------------------
    print("leitura df_QUERY_TELA_164_FOTO")
    df_QUERY_TELA_164_FOTO = pd.read_json(
        r'files\df_QUERY_TELA_164_FOTO.json', lines=True)

    if not df_QUERY_notasExport.empty:
        print("nota_export OK")
        print(df_QUERY_notasExport)
        colunas_atuais = df_QUERY_notasExport.columns
        df_QUERY_notasExport_filtred = df_QUERY_notasExport[(df_QUERY_notasExport['Status do sistema'] == 'MSPN') |
                                                            (df_QUERY_notasExport['Status do sistema'] == 'MSPR')]

        mapa_colunas = {
            'Data da nota': 'DT_NOTA',
            'Hora da nota': 'Hora da nota',
            'Equipam.': 'ATIVO_TL',
            'Parada': 'Flag',
            'Notificador': 'Notificador',
            'TpNota': 'TP_NOTA',
            'CódFalha': 'CodFalha',
            'Descrição': 'TEXTO',
            'Texto do item': 'Texto_Item',
            'Texto da causa': 'Texto_Causa',
            'Status do sistema': 'Aux_Status',
            'Nota': 'NOTA',
            'Novo Descrição': 'TEXTO',
            'Novo Texto do item': 'Texto_Item',
            'Novo Texto da causa': 'Texto_Causa',
            'Modelo': 'ATIVO_TL'
        }

        colunas_convertidas = [mapa_colunas.get(
            col, col) for col in colunas_atuais]

        if len(colunas_convertidas) == len(colunas_atuais):
            df_QUERY_notasExport_filtred.columns = colunas_convertidas
        else:
            print(
                "Erro: O número de colunas mapeadas não corresponde ao número de colunas no DataFrame.")

        df_QUERY_notasExport_filtred['BASE'] = "SAP_Arquivo_LOCAL"
        df_QUERY_z369['BASE'] = "SAP_STAGE"

        df_concatenado = pd.concat(
            [df_QUERY_z369, df_QUERY_notasExport_filtred])
        df_resultado_Export_z369_final = df_concatenado.drop_duplicates(
            subset=['NOTA'], keep='last')
        df_QUERY_z369 = df_resultado_Export_z369_final
    else:
        user_info = "Sem dados Locais - APENAS STAGE"
        print("nota_export NOK")

    if len(df_QUERY_TELA_164_FOTO) > 1000:
        print("df_QUERY_TELA_164_FOTO OK")
    else:
        print("df_QUERY_TELA_164_FOTO NOK")

    if len(df_QUERY_z369) > 1000:
        print("df_QUERY_z369 OK")
    else:
        print("df_QUERY_z369 NOK")
# ---------------------------------------------
    # Cria a coluna 'ATIVO_TL' na tabela df_QUERY_TELA_164_FOTO
    df_QUERY_TELA_164_FOTO['ATIVO_TL'] = df_QUERY_TELA_164_FOTO['VAGAO'].astype(
        str).str.zfill(7) + df_QUERY_TELA_164_FOTO['SERIE']

    # Garante que 'ATIVO_TL' em df_QUERY_z369 também seja do tipo string
    df_QUERY_z369['ATIVO_TL'] = df_QUERY_z369['ATIVO_TL'].astype(str)

    # Realiza a junção com how='left' para manter todos os registros de df_QUERY_TELA_164_FOTO
    df_stage_merge = pd.merge(
        df_QUERY_TELA_164_FOTO, df_QUERY_z369, on='ATIVO_TL', how='left')

    # Converte a coluna 'NR_OS' para string ou vazio, conforme necessário
    df_stage_merge['NR_OS'] = df_stage_merge['NR_OS'].apply(
        lambda x: str(int(x)) if pd.notnull(x) else '')
    df_stage_merge['key'] = df_stage_merge['NOTA'].astype(str) + \
        df_stage_merge['ATIVO_TL']

    # Remove as duplicatas baseadas na coluna 'NOTA', mantendo o primeiro registro
    df_stage_merge = df_stage_merge.drop_duplicates(
        subset='key', keep='first')

    # Converte 'DT_NOTA' para o formato desejado, lidando com erros
    df_stage_merge['DT_NOTA'] = pd.to_datetime(
        df_stage_merge['DT_NOTA'], dayfirst=True, errors='coerce').dt.strftime('%Y-%m-%d')

    # Garante que 'ATIVO_TL' sem notas também apareçam
    # Substitui valores NaN em colunas críticas para evitar inconsistências
    df_stage_merge = df_stage_merge.fillna({
        'NR_OS': '',
        'NOTA': '',
        'DT_NOTA': '',
        'Aux_Status': ''
    })

# ---------------------------------------------

    Pontuacao_trem = pd.pivot_table(
        df_stage_merge, index='NR_OS', columns='TP_NOTA', aggfunc='size', fill_value=0)
    Pontuacao_trem = Pontuacao_trem.reset_index()

    nr_os_unicos = df_stage_merge['NR_OS'].unique()
    Pontuacao_trem = Pontuacao_trem.set_index('NR_OS').reindex(
        nr_os_unicos, fill_value=0).reset_index()

    Pontuacao_trem = Pontuacao_trem.merge(
        df_stage_merge[['TREM', 'NR_OS']].drop_duplicates(), on='NR_OS', how='left')
    Pontuacao_trem = Pontuacao_trem.sort_values(
        by='M2', ascending=False).drop_duplicates().reset_index(drop=True)

    Pontuacao_trem = Pontuacao_trem[['TREM', 'M1', 'M2','M9','M6' , 'NR_OS']]
    Pontuacao_trem['TREM'] = Pontuacao_trem['TREM'].astype(str)
    Pontuacao_trem = Pontuacao_trem[(Pontuacao_trem['TREM'] != "None")]

    df = Pontuacao_trem

    print(df)
except Exception as e:
    print(f"Erro: {e}")

df_stage_merge = df_stage_merge.drop_duplicates(
    subset='key', keep='first')

df_stage_merge.to_json(
    'files\\df_stage_merge_saved.json', orient='records', lines=True)

# Apenas para teste, exportar CSVa
# df_stage_merge.to_csv('files//df_stage_merge.csv', index=False)

leitura df_QUERY_z369
Alguma coluna continha 'Unnamed'. Cabeçalhos substituídos com sucesso!
leitura df_QUERY_TELA_164_FOTO
nota_export OK
      Data da nota Hora da nota    Equipam. Parada Notificador TpNota  \
0       2024-12-03     06:50:30  0318345HPT    NaN     RICHARD     M1   
1       2024-12-03     05:28:34  3508714HTT    NaN    KEROLLYN     M1   
2       2024-12-03     05:28:34  0535443HFT    NaN    KEROLLYN     M1   
3       2024-12-03     03:14:57  0309478HPT      X     JOSEVAL     M2   
4       2024-12-03     05:48:22  3517284HTT    NaN    KEROLLYN     M2   
...            ...          ...         ...    ...         ...    ...   
13104   2024-01-02     21:44:58  6420150HNS    NaN      CEVADA     M1   
13105   2024-01-01     07:24:00  0300357HPT    NaN        GAVA     M1   
13106   2024-01-01     07:24:57  0300357HPT    NaN        GAVA     M1   
13107   2024-01-01     12:59:40  8401381HPT    NaN        GAVA     M1   
13108   2024-01-01     07:31:57  8403376HPT    NaN        

C:\Users\cs402807\AppData\Local\Temp\ipykernel_20008\2016980137.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_QUERY_notasExport_filtred['BASE'] = "SAP_Arquivo_LOCAL"
C:\Users\cs402807\AppData\Local\Temp\ipykernel_20008\2016980137.py:102: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_QUERY_z369['ATIVO_TL'] = df_QUERY_z369['ATIVO_TL'].astype(str)


    TREM   M1  M2  M9  M6    NR_OS
1    L90  104  13   0   0  1990222
2    C63  102  12   0   0  1990294
3    C67   96  12   0   1  1990615
4    L95   76  12   0   0  1990206
5    C74   82  12   0   0  1990607
..   ...  ...  ..  ..  ..      ...
210  M15    0   0   0   0  1989833
211  Z64    0   0   0   0  1990525
212  U98    0   0   0   0  1988839
213  I23  132   0   0   0  1990517
214  B05    0   0   0   0  1990532

[214 rows x 6 columns]


In [31]:
from flask import Flask, render_template, send_from_directory, request, redirect, url_for, Response
import os
from zipfile import ZipFile
import shutil
import subprocess
from werkzeug.utils import secure_filename
import pandas as pd
import numpy as np
import time
from datetime import datetime


In [37]:


geralCards = pd.read_csv('files//submitted_data.csv')
# Filtra o DataFrame removendo o item pelo ID
geralCards = geralCards[geralCards['NR_OS'] != 'XX']
print(geralCards.head(5))
geralCards.to_csv('files//submitted_data.csv', index=False)

     NR_OS TREM    ATIVO_TL             data_now              ID DESC_LOTACAO  \
1  1990384  I62  0317691HPT  06/12/2024 10:43:00  20241206104300    CARREGADO   
2  1990384  I62  0317691HPT  06/12/2024 10:43:00  20241206104300    CARREGADO   
3  1990384  I62  0317691HPT  06/12/2024 10:43:00  20241206104300    CARREGADO   
4  1990384  I62  0317691HPT  06/12/2024 10:43:00  20241206104300    CARREGADO   
5  1990384  I62  0317691HPT  06/12/2024 10:42:26  20241206104226    CARREGADO   

         NOTA Pode_ir_para_TOM Obrigatoria Local_Retencao  ... Sequencia  \
1  23833883.0              nao         sim            ZRO  ...       3.0   
2  23836086.0              nao         sim            ZRO  ...       3.0   
3  23873486.0              nao         sim            ZRO  ...       3.0   
4  23861600.0              nao         sim            ZRO  ...       3.0   
5  23833883.0              nao         sim            ZRO  ...       3.0   

      DT_NOTA                                    TEXTO  

In [ ]:
import pandas as pd
from datetime import datetime

print("semretencao")
# Carrega o DataFrame existente
geralCards = pd.read_csv('files//submitted_data.csv')

# Obtém a data e hora atual
data_atual = datetime.now()



# Cria um novo DataFrame com os valores necessários
new_row = pd.DataFrame({
    'NR_OS': ["XX"],
    'TREM': ["XX"],
    'ATIVO_TL': ["Sem Retenções"],
    'data_now': [data_atual],  # Adiciona a data diretamente como datetime
})

# Cria o ID usando a data e hora no formato desejado


new_row['ID'] = new_row['data_now'].dt.strftime('%Y%m%d%H%M%S')
new_row['data_now'] = new_row['data_now'].dt.strftime('%d/%m/%Y %H:%M:%S')

# Concatena com o DataFrame original
geralCards = pd.concat([new_row, geralCards], ignore_index=True)

# Salva o resultado em CSV
#geralCards.to_csv('files//submitted_data.csv', index=False)

# Exibe os 5 primeiros registros
geralCards.head(5)


semretencao


,NR_OS,TREM,ATIVO_TL,data_now,ID,DESC_LOTACAO,NOTA,Pode_ir_para_TOM,Obrigatoria,Local_Retencao,...,Sequencia,DT_NOTA,TEXTO,Texto_Causa,TP_NOTA,Local,Observacao_Retencao,Qtd_vg_Por_OS,DataHoraPassagem,data_now2
0,XX,XX,Sem Retenções,2024-12-06 10:48:50.773243,20241206104850,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1990909,R87,3507726HTT,06/12/2024 10:44:12,20241206104412,VAZIO,23907265.0,nao,sim,ZRO,...,85.0,2024-11-27,RD|PAD|PAD|DEFORMADO|,PAD Trincado 1E 2E 3D 4D TRKV,M1,ZRO,NaN,83.0,,2024-12-06 10:44:12
2,1990909,R87,3507726HTT,06/12/2024 10:44:12,20241206104412,VAZIO,23882436.0,nao,sim,ZRO,...,85.0,2024-11-03,RD|RODA|WCM|IMPACTO MÉDIO,225kN|3D|||,M2,ZRO,NaN,83.0,,2024-12-06 10:44:12
3,1990909,R87,8404861HPT,06/12/2024 10:44:12,20241206104412,VAZIO,23868260.0,nao,sim,ZRO,...,15.0,2024-10-12,RD|RODA|WCM|IMPACTO MÉDIO,||239kN|4D E 3E|||,M2,ZRO,NaN,83.0,,2024-12-06 10:44:12
4,1990909,R87,8420769HFT,06/12/2024 10:44:12,20241206104412,VAZIO,23638445.0,nao,sim,ZRO,...,23.0,2024-11-21,RD|RODA|WCM|IMPACTO MÉDIO,||229kN|4E|||,M2,ZRO,NaN,83.0,,2024-12-06 10:44:12


In [ ]:
print("email")
# Carregar o arquivo CSV
geralCards = pd.read_csv('files//submitted_data.csv')

# Filtrar apenas as linhas com TP_NOTA == 'M2'
geralCards_Email = geralCards[
    (geralCards['TP_NOTA'].isin(['M2', 'M9', 'M6'])) | 
    (geralCards['Observacao_Retencao'].notna())
]

# Obter o parâmetro `id` da URL e converter para uma lista
OSfilters = [1971765]


# Filtrando o DataFrame com os IDs fornecidos
geralCards_Email = geralCards_Email[geralCards_Email['NR_OS'].isin(
    OSfilters)]

geralCards_Email['Defeitos'] = geralCards_Email['TEXTO'].astype(str) + \
    " " + geralCards_Email['Texto_Causa'].astype(str)

# Agrupar por 'NR_OS' e concatenar os defeitos
result = geralCards_Email.groupby(['NR_OS', 'ATIVO_TL'], as_index=False).agg({
    'TREM': 'first',  # Manter o primeiro valor
    'Qtd_vg_Por_OS': 'first',
    'DataHoraPassagem' : 'first',
    'NOTA': 'first',  # Manter o primeiro valor
    'Pode_ir_para_TOM': 'first',
    'Obrigatoria': 'first',
    'Local': 'first',
    'Isolado': 'first',
    'Rec_Sistema': 'first',
    'Sequencia': 'first',
    'DT_NOTA': 'first',
    # Concatenar os valores de 'Defeitos'
    'Defeitos': lambda x: ' + '.join(x.astype(str)),
    'Observacao_Retencao':'first'
    'data_now': 'first'  # Manter o primeiro valor
})
# Depurar o número de resultados
print(len(result))

email
1


In [ ]:
#geralCards['Sequencia'] = geralCards['Sequencia'].astype(str).replace('.0', '')

geralCards['Sequencia']

0     85
1     85
2     15
3     23
4      2
5      2
6      2
7      2
8      2
9      4
10    35
11    35
12    35
13    35
14    13
15    13
16    13
17    13
18    20
19    20
20    20
21    20
22    20
23    20
24    20
25    11
26    11
27    13
28    13
29    13
30    13
31    13
32    13
33    13
34    13
35    13
36    13
Name: Sequencia, dtype: int32

In [17]:

result['categoria'] = result['NR_OS'].ne(result['NR_OS'].shift()).cumsum().apply(lambda x: 'A' if x % 2 == 0 else 'B')
result

,NR_OS,ATIVO_TL,TREM,Qtd_vg_Por_OS,DataHoraPassagem,NOTA,Pode_ir_para_TOM,Obrigatoria,Local,Isolado,Rec_Sistema,Sequencia,DT_NOTA,Defeitos,data_now,categoria
0,1990081,8413991HFT,Y46,123,05/12/2024 11:33,23886221.0,nao,sim,ZRO,nao,sim,13.0,2024-11-07,CCT|ENGATE|ROTOR|QUEBRADO |CB||||RAF 9048|,05/12/2024 14:43:22,B
1,1990081,8424250HFT,Y46,123,05/12/2024 11:33,23912456.0,nao,sim,ZRO,nao,sim,20.0,2024-12-04,CCT|ENGATE|ROTOR|QUEBRADO |CB||||RAF 9177|,05/12/2024 14:43:22,B
2,1990615,0608220PCT,C67,58,06/12/2024 14:12,23851388.0,nao,sim,ZRO,nao,sim,11.0,2024-09-21,REVI CRITICO R1E roda cascalada,05/12/2024 14:12:30,A
3,1990615,0800694PCT,C67,58,06/12/2024 14:12,23875121.0,nao,sim,ZRO,nao,sim,13.0,2024-10-23,CCT|ENGATE|ENGATE|QUEBRADO| |CB||||15mm ENGATE...,05/12/2024 14:12:30,A


In [ ]:
df_stage_merge_tela1 = df_stage_merge
valor_filtro = '1990089'
# Filtra o DataFrame com base no valor fornecido
df_stage_merge_tela1_filtred = df_stage_merge_tela1[df_stage_merge_tela1['NR_OS'] == valor_filtro]
df_stage_merge_tela1_filtred['TP_NOTA'].value_counts()
#df_stage_merge_tela1_filtred



df_stage_merge_tela1

In [ ]:
df_stage_merge_tela_bkp = df_stage_merge_tela1

In [ ]:
df_stage_merge = pd.read_json(
    'files\\df_stage_merge_saved.json', orient='records', lines=True)

valor_filtro = '1990089'

# Filtra o DataFrame com base no valor fornecido
df_filtrado = df_stage_merge[df_stage_merge['NR_OS'] == valor_filtro]
print(len(df_filtrado))
print(df_filtrado.columns)
df_stage_merge_tela1_filtred.head()


0
Index(['VAGAO', 'ID_VAGAO', 'SERIE', 'TREM', 'SEQUENCIA', 'LOCAL', 'ORIGEM',
       'DESTINO', 'NR_OS', 'DT_CARGA', 'COD_LINHA', 'ATIVO_TL', 'NOTA',
       'DT_NOTA', 'Notificador', 'Texto_Causa', 'TP_NOTA', 'TEXTO',
       'Texto_Item', 'Texto_AVARIA', 'DATA_QMDAT', 'CodFalha', 'Flag',
       'Aux_Status', 'BASE', 'Hora da nota', 'Prioridade', 'Sub-Trecho',
       'Centro', 'Centro trab.respons.'],
      dtype='object')


,VAGAO,ID_VAGAO,SERIE,TREM,SEQUENCIA,LOCAL,ORIGEM,DESTINO,NR_OS,DT_CARGA,COD_LINHA,ATIVO_TL,NOTA,DT_NOTA,Notificador,Texto_Causa,TP_NOTA,TEXTO,Texto_Item,Texto_AVARIA,DATA_QMDAT,CodFalha,Flag,Aux_Status,BASE,Hora da nota,Prioridade,Sub-Trecho,Centro,Centro trab.respons.
161,801275,341179,PCT,C75,25.0,ZUE,ZTG,TRO,1990089,1733121421000,None,0801275PCT,23721857.0,28/03/2024,FLAVIANE,nao manten aplicação do freio,M1,REVI MONITORADO,*Freio:Outros*,*Freio:Outros*,20240328.0,VGFO,,Pendente,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
245,580058,350142,PCT,C75,6.0,ZUE,ZTG,TRO,1990089,1733121421000,None,0580058PCT,23638406.0,25/12/2023,WILLIAN,mandibula CA / CB,M1,REVI MONITORADO,*CCT:Mandíbula*,*CCT:Mandíbula*,20231225.0,VGJM,,Pendente,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
489,608157,351436,PCT,C75,21.0,ZUE,ZTG,TRO,1990089,1733121421000,None,0608157PCT,23263432.0,20/08/2022,CASSIO,Sem telha suporte do engate Cab B,M1,REVI MONITORADO,*CCT:Telha do Engate*,*CCT:Telha do Engate*,20220820.0,VGJJ,,Pendente,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
490,608157,351436,PCT,C75,21.0,ZUE,ZTG,TRO,1990089,1733121421000,None,0608157PCT,23212043.0,10/06/2022,SHEILLA,S/chapa de desgaste apoio do engate C/B,M1,REVI MONITORADO,*CCT:Apoio do Engate*,*CCT:Apoio do Engate*,20220610.0,VGAC,,Pendente,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
491,608157,351436,PCT,C75,21.0,ZUE,ZTG,TRO,1990089,1733121421000,None,0608157PCT,23449979.0,01/05/2023,SHEILLA,Mandíbula C/A monitorar,M2,REVI CRÍTICO,Quebra mandíbula engate,Quebra mandíbula engate,20230501.0,VGJM,,Pendente,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
492,608157,351436,PCT,C75,21.0,ZUE,ZTG,TRO,1990089,1733121421000,None,0608157PCT,23819319.0,01/05/2023,SHEILLA,Mandíbula C/A monitorar,M1,REVI MONITORADO,Quebra mandíbula engate,Quebra mandíbula engate,20230501.0,VGJM,,Pendente,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
521,607134,351187,PCT,C75,27.0,ZUE,ZTG,TRO,1990089,1733121421000,None,0607134PCT,23238197.0,18/07/2022,SHEILLA,Corrente freio manual quebrada,M1,FREIO|FREIO MANUAL|CORRENTE|QUEBRADA,*Freio Manual:Corrente*,*Freio Manual:Corrente*,20220718.0,VGFF,,Pendente,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
522,607134,351187,PCT,C75,27.0,ZUE,ZTG,TRO,1990089,1733121421000,None,0607134PCT,23248210.0,02/08/2022,PAOLA,Corrente do manual Avariado CB,M1,FREIO|FREIO MANUAL|CORRENTE|QUEBRADA,*Freio Manual:Corrente*,*Freio Manual:Corrente*,20220802.0,VGFF,,Pendente,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
523,607134,351187,PCT,C75,27.0,ZUE,ZTG,TRO,1990089,1733121421000,None,0607134PCT,23256458.0,11/08/2022,PAOLA,Corrente quebrada,M1,FREIO|FREIO MANUAL|CORRENTE|QUEBRADA,*Freio Manual:Corrente*,*Freio Manual:Corrente*,20220811.0,VGFF,,Pendente,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
524,607134,351187,PCT,C75,27.0,ZUE,ZTG,TRO,1990089,1733121421000,None,0607134PCT,23819318.0,01/05/2023,SHEILLA,Mandíbula C/B monitorar,M1,REVI MONITORADO,Quebra mandíbula engate,Quebra mandíbula engate,20230501.0,VGJM,,Pendente,SAP_STAGE,NaN,NaN,NaN,NaN,NaN


In [49]:
df_stage_merge['TP_NOTA'].value_counts()

M1    320
M2     35
Name: TP_NOTA, dtype: int64

In [10]:
df_QUERY_z369

,NOTA,DT_NOTA,Notificador,Texto_Causa,TP_NOTA,TEXTO,Texto_Item,Texto_AVARIA,DATA_QMDAT,CodFalha,Flag,Aux_Status,ATIVO_TL,BASE,Hora da nota,Prioridade,Sub-Trecho,Centro,Centro trab.respons.
0,23550115,31/08/2023,IGOR,Tremonha/ Bica/ Quadro de tremonha,M1,KM 1491+657 TREM W01,Tremonha/ Bica/ Quadro de tremonha,Tremonha/ Bica/ Quadro de tremonha,20230831.0,VGCT,,Pendente,0984426HNS,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
1,23550522,31/08/2023,PAOLA,C/A Chapa da braçadeira arriada,M1,REVI MONITORADO,Abraçadeira do engate avariada/ quebrada,Abraçadeira do engate avariada/ quebrada,20230831.0,VGJB,,Em processamento,0531073HPT,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
2,23605042,08/11/2023,NATHALIA,RODA FRIA (CW),M1,KM 000+000 TREM R32,Não aplica freio - roda fria,Não aplica freio - roda fria,20231108.0,VGFH,,Em processamento,3019713HPT,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
3,23546621,26/08/2023,SHEILLA,c/a apoio trincado,M1,REVI MONITORADO,Suporte/Apoio do engate trincado/ quebra,Suporte/Apoio do engate trincado/ quebra,20230826.0,VGAC,,Pendente,0531430HPT,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
4,23605149,08/11/2023,RICARDO,CA olhal superior do engate trincado,M1,CCT|ENGATE|OLHAL|TRINCADO|,*CCT:Olhal do Engate*,*CCT:Olhal do Engate*,20231108.0,VGJC,,Pendente,0304131HPT,SAP_STAGE,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10887,23767953,2024-06-01 00:00:00,FLAVIANE,CB le 2 molas carga solida,M1,REVI MONITORADO,*Amort:Mola*,NaN,NaN,VGTM,NaN,MSPN,8401322HPT,SAP_Arquivo_LOCAL,03:42:54,1.0,PCZ-ZPG,NaN,NaN
10888,23768120,2024-06-01 00:00:00,SHEILLA,CB L/E 1 mola carga sólida,M1,TR|PACOTE DE MOLA|MOLA|MARCA SOLIDA,*Amort:Mola*,NaN,NaN,VGTM,NaN,MSPN,8413304HFT,SAP_Arquivo_LOCAL,19:48:23,1.0,PCZ-ZPG,NaN,NaN
10889,23768157,2024-06-01 00:00:00,SHEILLA,R4 LD escoamento de material,M1,REVI MONITORAMENTO,*Rd:Pista*,NaN,NaN,VGHB,NaN,MSPN,0559288HFT,SAP_Arquivo_LOCAL,21:41:58,1.0,PCZ-ZPG,NaN,NaN
10890,23767939,2024-06-01 00:00:00,KEROLLYN,Roda fria (CW),M1,KM 000+000 TREM I48,Não aplica freio - roda fria,NaN,NaN,VGFH,NaN,MSPN,8417628HFT,SAP_Arquivo_LOCAL,04:59:14,3.0,NaN,NaN,NaN


In [ ]:
# Obtém o valor do ID da query string
# df_stage_merge = pd.read_csv('files//df_stage_merge.csv')
df_stage_merge = pd.read_json(
    'files\\df_stage_merge_saved.json', orient='records', lines=True)


valor_filtro = '1990089'



# Filtra o DataFrame com base no valor fornecido
df_filtrado = df_stage_merge[df_stage_merge['NR_OS'] == valor_filtro]
print(len(df_filtrado))
print(df_filtrado.columns)

print("index count")
print(df_filtrado['TP_NOTA'].value_counts())

# Processa os dados filtrados (pivot table)
df_filtrado['TP_NOTA'] = df_filtrado['TP_NOTA'].fillna("Sem nota")
print("A")

b_resumo = pd.pivot_table(
    df_filtrado, index='ATIVO_TL', columns='TP_NOTA', aggfunc='size', fill_value=0)
print("b")
print(b_resumo.columns)
b_resumo = b_resumo.reset_index()
print("c")
b_resumo = b_resumo.merge(
    df_filtrado[['ATIVO_TL', 'SEQUENCIA']], on='ATIVO_TL', how='left')
print("d")
b_resumo = b_resumo.sort_values(
    by='SEQUENCIA').drop_duplicates().reset_index()
print("e")
print(len(df_filtrado[df_filtrado['TP_NOTA'] == 'M2']))

if len(df_filtrado[df_filtrado['TP_NOTA'] == "M2"]) == 0:
    print("entrou")
    b_resumo["M2"] = 0
if len(df_filtrado[df_filtrado['TP_NOTA'] == "M1"]) == 0:
    print("entrou")
    b_resumo["M1"] = 0

b_resumo = b_resumo[['ATIVO_TL', 'M1', 'M2', 'SEQUENCIA']]
b_resumo['SEQUENCIA'] = b_resumo['SEQUENCIA'].astype(int).astype(str)



192
Index(['VAGAO', 'ID_VAGAO', 'SERIE', 'TREM', 'SEQUENCIA', 'LOCAL', 'ORIGEM',
       'DESTINO', 'NR_OS', 'DT_CARGA', 'COD_LINHA', 'ATIVO_TL', 'NOTA',
       'DT_NOTA', 'Notificador', 'Texto_Causa', 'TP_NOTA', 'TEXTO',
       'Texto_Item', 'Texto_AVARIA', 'DATA_QMDAT', 'CodFalha', 'Flag',
       'Aux_Status', 'BASE', 'Hora da nota', 'Prioridade', 'Sub-Trecho',
       'Centro', 'Centro trab.respons.'],
      dtype='object')
index count
Series([], Name: TP_NOTA, dtype: int64)
A
b
Index(['Sem nota'], dtype='object', name='TP_NOTA')
c
d
e
0
entrou
entrou


C:\Users\leona\AppData\Local\Temp\ipykernel_23176\1947950475.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtrado['TP_NOTA'] = df_filtrado['TP_NOTA'].fillna("Sem nota")


In [26]:
df_filtrado['TP_NOTA'].value_counts()

Sem nota    192
Name: TP_NOTA, dtype: int64

In [ ]:
df_filtrado['TP_NOTA'].value_counts()

Sem nota    192
Name: TP_NOTA, dtype: int64

In [27]:
df_filtrado

,VAGAO,ID_VAGAO,SERIE,TREM,SEQUENCIA,LOCAL,ORIGEM,DESTINO,NR_OS,DT_CARGA,...,DATA_QMDAT,CodFalha,Flag,Aux_Status,BASE,Hora da nota,Prioridade,Sub-Trecho,Centro,Centro trab.respons.
136,801275,341179,PCT,C75,25.0,ZUE,ZTG,TRO,1990089,1733121421000,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
215,580058,350142,PCT,C75,6.0,ZUE,ZTG,TRO,1990089,1733121421000,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
430,608157,351436,PCT,C75,21.0,ZUE,ZTG,TRO,1990089,1733121421000,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
452,607134,351187,PCT,C75,27.0,ZUE,ZTG,TRO,1990089,1733121421000,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
526,607142,351009,PCT,C75,20.0,ZUE,ZTG,TRO,1990089,1733121421000,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43051,573451,344860,PCT,C75,29.0,ZUE,ZTG,TRO,1990089,1733121421000,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
43099,607134,351187,PCT,C75,27.0,ZUE,ZTG,TRO,1990089,1733121421000,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
43280,607151,351158,PCT,C75,30.0,ZUE,ZTG,TRO,1990089,1733121421000,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
43427,800716,340869,PCT,C75,17.0,ZUE,ZTG,TRO,1990089,1733121421000,...,NaN,None,None,None,None,None,NaN,None,NaN,NaN
